In [2]:
import numpy as np
import pandas as pd
import os
import glob
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import psycopg2
from sqlalchemy import create_engine

# Read in Datasets

In [3]:
folder_path = "data/NABBP_Lookups_2022" 
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        # drop the first column of every csv file
        if len(df.columns) > 0:
            df = df.iloc[:, 1:]
        # Convert column names to lowercase for sql
        df.columns = df.columns.str.lower()
        df_name = os.path.splitext(filename)[0]
        globals()[df_name] = df

# abbreviation transform

In [4]:
def abbrev_to_state(abbrev):
    state_dict = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 
        'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 
        'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 
        'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 
        'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 
        'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 
        'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 
        'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 
        'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
    }
    
    return state_dict.get(abbrev.upper(), "Invalid abbreviation")

In [38]:
folder_path = 'data/band/'
csv_files = glob.glob(folder_path + 'NABBP_2022_grp_09.csv')
for file_path in csv_files:
    num = int(file_path.split('_')[-1].split('.')[0])
    df = pd.read_csv(file_path)
    # Convert column names to lowercase
    df.columns = df.columns.str.lower()
    # Select only the columns we need
    df = df[['band', 'event_year', 'event_month','event_day', 'iso_country', 'iso_subdivision', 'lat_dd', 'lon_dd', 'age_code', 'sex_code', 'species_id']]
    df = df[df['iso_country'] == 'US']
    df = df.drop('iso_country', axis=1)
    df['iso_subdivision'] = df['iso_subdivision'].apply(lambda x: x[3:])
    df['iso_subdivision'] = df['iso_subdivision'].apply(lambda x: abbrev_to_state(x))
    df_name = f'df_{num}'
    globals()[df_name] = df # only choose US data

/tmp/ipykernel_125/888025925.py:5: DtypeWarning: Columns (2,13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [18]:
#df = pd.read_csv('df_temp.csv')
#df_temp = df_1
#df_temp = df_1

In [41]:
df_temp['species_id'].unique()

array([1690, 1691, 1695, 1698, 1699, 1700, 1703, 1710, 1719, 1760, 1770,
       1780, 1440, 1400, 1350, 1360, 1370, 1392, 1410, 1420, 1430, 1321,
       1322, 1323, 1325, 1326, 1327, 1330, 1331, 1337, 1340, 1380, 1390,
       1401, 1431])

In [39]:
df_temp = pd.concat([df_temp, df], ignore_index=True)

In [40]:
len(df_temp)

5810276

In [9]:
#df_temp.to_csv('df_temp.csv', index=False)

In [43]:
del df_temp

In [42]:
df = df_temp

In [44]:
species = species.drop('count(1)', axis=1)

In [57]:
df.dtypes

band                object
event_year           int64
event_month          int64
event_day            int64
iso_subdivision     object
lat_dd             float64
lon_dd             float64
age_code           float64
sex_code           float64
species_id           int64
eventcode           object
dtype: object

In [55]:
df['eventcode'] = df['event_month'].astype(str) + '/' + df['event_day'].astype(str)

In [6]:
species.dtypes

species_id        int64
alpha_code       object
species_name     object
sci_name         object
recomendsize     object
allowablesize    object
endangered       object
raptor           object
gamebird         object
bbl_grp          object
dtype: object

In [7]:
sex.dtypes

sex_code            int64
sex_description    object
dtype: object

In [8]:
age.dtypes

age_code             int64
age_alpha_code      object
age_description     object
age_explain_text    object
dtype: object

In [52]:
inexact_dates.rename(columns={'event_month/event_day': 'eventcode'}, inplace=True)

In [53]:
inexact_dates.dtypes

eventcode                   object
inexact_date_description    object
dtype: object

# Create a database

In [46]:
dbserver = psycopg2.connect(
    host = 'postgres',
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    port = 5432
)
dbserver.autocommit = True

In [47]:
cursor = dbserver.cursor()

In [48]:
try:
    cursor.execute('CREATE DATABASE bird')
except:
    cursor.execute('DROP DATABASE bird')
    cursor.execute('CREATE DATABASE bird')

In [49]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'bird'
))

In [56]:
df.to_sql('bird', con=engine, index=False, chunksize=2000, if_exists='replace')

2905276

In [54]:
species.to_sql('species', con=engine, index=False, chunksize=1000, if_exists='replace')
sex.to_sql('sex', con=engine, index=False, chunksize=1000, if_exists='replace')
age.to_sql('age', con=engine, index=False, chunksize=1000, if_exists='replace')
inexact_dates.to_sql('inexact_dates', con=engine, index=False, chunksize=1000, if_exists='replace')

61